# Run this cell

In [50]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
import numpy as np
import pandas as pd
from matplotlib.collections import PolyCollection
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import random
import sys, os

ROOT_PATH = os.path.abspath("..").split("data_processing")[0]
print("ROOT", ROOT_PATH)
module_path = os.path.abspath(os.path.join(ROOT_PATH+"/utils/"))
if module_path not in sys.path:
    print("appended")
    sys.path.append(module_path)

import functions as f

module_path = os.path.abspath(os.path.join(ROOT_PATH+"/data_processing/"))
if module_path not in sys.path:
    print("appended")
    sys.path.append(module_path)


%autoreload
import wt_data
import ff_transform
import explore_fft_time_data

ROOT /Users/stianismar/Dropbox/gitProsjekter/master-thesis/src


# Run from here

In [52]:
COMPONENT_NAME = 'GbxHssRr'

In [57]:
'''
def load_wt(wt_name):
    wt_instance = wt_data.load_instance(wt_name, load_minimal=False)
    down = []
    times = []
    number_of_intervals = len(wt_instance.ten_second_intervals)
    for i,interval in enumerate(wt_instance.ten_second_intervals):
        if i <= 5:
            vibration_signal = interval.sensor_df['GbxHssRr;0,0102;m/s2']
            time = interval.sensor_df['TimeStamp']
            times.append(time)
            # down.append((vibration_signal[::100]))
            down.append(vibration_signal)
        if i > (number_of_intervals - 5):
            print("Appending the last interval")
            vibration_signal = interval.sensor_df['GbxHssRr;0,0102;m/s2']
            down.append(vibration_signal)

            time = interval.sensor_df['TimeStamp']
            times.append(time)
    del wt_instance
    del interval
    return down, times

vib_signal_wt4, times_wt4 = load_wt("WTG04")

from numpy import savez_compressed
savez_compressed('../../sensor_data/vib_signal_wt4.npz', vib_signal_wt4)
savez_compressed('../../sensor_data/times_wt4.npz', times_wt4)

del vib_signal_wt4, times_wt4
'''

'\ndef load_wt(wt_name):\n    wt_instance = wt_data.load_instance(wt_name, load_minimal=False)\n    down = []\n    times = []\n    number_of_intervals = len(wt_instance.ten_second_intervals)\n    for i,interval in enumerate(wt_instance.ten_second_intervals):\n        if i <= 5:\n            vibration_signal = interval.sensor_df[\'GbxHssRr;0,0102;m/s2\']\n            time = interval.sensor_df[\'TimeStamp\']\n            times.append(time)\n            # down.append((vibration_signal[::100]))\n            down.append(vibration_signal)\n        if i > (number_of_intervals - 5):\n            print("Appending the last interval")\n            vibration_signal = interval.sensor_df[\'GbxHssRr;0,0102;m/s2\']\n            down.append(vibration_signal)\n\n            time = interval.sensor_df[\'TimeStamp\']\n            times.append(time)\n    del wt_instance\n    del interval\n    return down, times\n\nvib_signal_wt4, times_wt4 = load_wt("WTG04")\n\nfrom numpy import savez_compressed\nsavez_comp

In [54]:
from numpy import load

# Load compressed data
vib_signal_wt4 = load('../../sensor_data/vib_signal_wt4.npz')
times_wt4 = load('../../sensor_data/times_wt4.npz')

In [55]:
vib_signal_wt4 = vib_signal_wt4['arr_0']
times_wt4 = times_wt4['arr_0']

In [56]:
print(vib_signal_wt4.shape)
print(times_wt4.shape)

(10, 262144)
(10, 262144)


In [ ]:
from scipy import stats
stats.describe(vib_signal_wt4[2])

In [ ]:
def plot_vib_consecutive(down):
    indexes = []
    index_start=0
    fig, ax = plt.subplots(figsize=(15,5))
    for i, d in enumerate(down):
        indexes.append(index_start)
        index_end = len(d)
        x = np.arange(index_start, index_start+index_end)
        index_start = len(x) + index_start
        plt.plot(x,d)
        plt.margins(0)
    print(len(np.linspace(0,index_start+index_end,5)))
    plt.xticks(np.linspace(0,index_start,24),[int(round(elem)) for i,elem in enumerate(np.linspace(1,len(down)+1,24))])
    #locs, labels = plt.xticks()
    #ax.set_xticks(indexes)
    #ax.set_xticklabels(np.arange(1,len(indexes)+1))
    # plt.xticks(fontsize=7, rotation=90)
    print((indexes[-1]))
    plt.title("Gearbox HSS")
    plt.xlabel("Interval number")
    plt.ylabel("Vib Amplitude")
    plt.show()
plot_vib_consecutive(vib_signal_wt4)

In [ ]:
def get_sampling_freq(signal,times):
    for i in range(len(np.array(times))):
        times = np.array(times)
        T = times[0] - times[1]
        ending_time = np.array(times)[-1]
        N = len(signal)
        fs = N/ending_time
        break
    return fs, N, T, ending_time

fs, N, T, ending_time = get_sampling_freq(vib_signal_wt4[0],times_wt4[0])
fs

In [ ]:
def perform_fft(amplitudes, timestamps,ax ):
    t = timestamps
    sig = amplitudes
    mean_amplitude = np.mean(sig)
    sig = sig - mean_amplitude  # Centering around 0
    fft = np.fft.fft(sig)
    N = sig.size
    T = t[1] - t[0]

    f = np.linspace(0, 1 / T, N, )  # start, stop, number of. 1 / T = frequency is the biggest freq

    f = f[:N // 2]
    y = np.abs(fft)[:N // 2]
    y_norm = np.abs(fft)[:N // 2] * 1 / N  # Normalized
    fft_modulus_norm = y_norm
    ax.plot(f,fft_modulus_norm)

# perform_fft(sig, t)

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt
def sine_generator(fs, sinefreq, duration):
    T = duration
    nsamples = fs * T
    w = 2. * np.pi * sinefreq
    t_sine = np.linspace(0, T, nsamples, endpoint=False)
    y_sine = np.sin(w * t_sine)
    result = pd.DataFrame({ 
        'data' : y_sine} ,index=t_sine)
    return result

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

# fps = 2000
#sine_fq = 10 #Hz
duration = ending_time #seconds
#sine_5Hz = sine_generator(fps,sine_fq,duration)
#sine_fq = 1 #Hz
duration = ending_time #seconds
# sine_1Hz = sine_generator(fps,sine_fq,duration)

# sine = sine_5Hz + sine_1Hz
hp_cutoff = 2000
filtered_sine_wt4 = butter_highpass_filter(vib_signal_wt4[0],hp_cutoff,fs)
filtered_sine_wt4

## Butter bandpass

In [ ]:
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

## Plot_enveloping_process

In [ ]:
def plot_enveloping_process(rows, cols, COMPONENT_NAME, raw_signal, filtred_signal, timestamps, lowcut, highcut):
    filtred_signal
    fig, ax = plt.subplots(rows, cols)
    fig.suptitle(f'{COMPONENT_NAME}: Envelop signal processing ',fontsize=16, y=1.05)

    ax[0,0].plot(timestamps,raw_signal)
    ax[0,0].set_title(f"Raw vibration signal from {COMPONENT_NAME} ")
    perform_fft(raw_signal,timestamps,ax[0,1])
    ax[0,1].set_xlabel("Frequency [Hz]")
    ax[0,1].set_title(f"Raw signal from {COMPONENT_NAME}")

    ax[1,0].plot(timestamps,filtred_signal)
    ax[1,0].set_title(f'filtered signal with lower cutoff {lowcut} and higher {highcut} Hz')
    perform_fft(filtred_signal, timestamps, ax[1,1])
    ax[1,1].set_xlabel("Frequency [Hz]")
    ax[1,1].set_xlim(lowcut-500,highcut+500)
    plt.tight_layout()
    plt.margins(0)

## lowcut = 4000, highcut = 6000
### First interval

In [ ]:
lowcut = 4000
highcut = 6000
fs = fs # already defined
bandpass1_filtered = butter_bandpass_filter(vib_signal_wt4[0],lowcut, highcut, fs, order=5)
bandpass1_filtered
plot_enveloping_process(2,2,COMPONENT_NAME, vib_signal_wt4[0],bandpass1_filtered,times_wt4[0],lowcut,highcut )

### Last interval

In [ ]:
lowcut = 4000
highcut = 6000
fs = fs # already defined
bandpass1_filtered = butter_bandpass_filter(vib_signal_wt4[-1],lowcut, highcut, fs, order=5)
bandpass1_filtered
plot_enveloping_process(2,2,COMPONENT_NAME, vib_signal_wt4[-1],bandpass1_filtered,times_wt4[-1],lowcut,highcut )

df = pd.DataFrame(bandpass2_filtered)
df['timestamp'] = times_wt4[-1]
# df.plot.hist(bins=2000)
df.iloc[60:800].plot.line(x='timestamp',y=0,title="Zoomed in on index 60 to 800 in Filtered signal",legend=False)

### Second to last interval

In [ ]:
stl_index = len(times_wt4) - 2 # Second to last
print(vib_signal_wt4.shape)
lowcut = 4000
highcut = 6000
fs = fs # already defined
bandpass2_filtered = butter_bandpass_filter(vib_signal_wt4[stl_index],lowcut, highcut, fs, order=5)
bandpass2_filtered
plot_enveloping_process(2,2,COMPONENT_NAME, vib_signal_wt4[stl_index],bandpass2_filtered,times_wt4[stl_index],lowcut,highcut)

df = pd.DataFrame(bandpass2_filtered)
df['timestamp'] = times_wt4[stl_index]
# df.plot.hist(bins=2000)
df.iloc[100:800].plot.line(x='timestamp',y=0,title="Zoomed in on index 60 to 800 in Filtered signal",legend=False)

## lowcut = 5000, highcut = 6000

### First interval

In [ ]:
lowcut = 5000
highcut = 6000
fs = fs # already defined
bandpass2_filtered = butter_bandpass_filter(vib_signal_wt4[0],lowcut, highcut, fs, order=5)
bandpass2_filtered
plot_enveloping_process(2,2,COMPONENT_NAME, vib_signal_wt4[0],bandpass2_filtered,times_wt4[0],lowcut,highcut )

df = pd.DataFrame(bandpass2_filtered)
df['timestamp'] = times_wt4[0]
# df.plot.hist(bins=2000)
df.iloc[100:800].plot.line(x='timestamp',y=0,title="Zoomed in on index 60 to 800 in Filtered signal",legend=False)

### Last interval

In [ ]:
lowcut = 5000
highcut = 6000
fs = fs # already defined
bandpass2_filtered = butter_bandpass_filter(vib_signal_wt4[-1],lowcut, highcut, fs, order=5)
bandpass2_filtered
plot_enveloping_process(2,2,COMPONENT_NAME, vib_signal_wt4[-1],bandpass2_filtered,times_wt4[-1],lowcut,highcut)

df = pd.DataFrame(bandpass2_filtered)
df['timestamp'] = times_wt4[-1]
# df.plot.hist(bins=2000)
df.iloc[100:800].plot.line(x='timestamp',y=0,title="Zoomed in on index 60 to 800 in Filtered signal",legend=False)

### Second to last interval Bandpass (5000, 6000 Hz)

In [ ]:
stl_index = len(times_wt4) - 2 # Second to last
print(vib_signal_wt4.shape)
lowcut = 5000
highcut = 6000
fs = fs # already defined
bandpass2_filtered = butter_bandpass_filter(vib_signal_wt4[stl_index],lowcut, highcut, fs, order=5)
bandpass2_filtered
plot_enveloping_process(2,2,COMPONENT_NAME, vib_signal_wt4[stl_index],bandpass2_filtered,times_wt4[stl_index],lowcut,highcut)


df = pd.DataFrame(bandpass2_filtered)
df['timestamp'] = times_wt4[stl_index]
# df.plot.hist(bins=2000)
df.iloc[100:800].plot.line(x='timestamp',y=0,title="Zoomed in on index 60 to 800 in Filtered signal",legend=False)

## Another type of filter

In [ ]:
def hp_filter(amplitudes, lower_freq_limit):
    sig = amplitudes
    fig, (ax2) = plt.subplots(1, 1, sharex=True)
    sos = signal.butter(10, lower_freq_limit, 'hp', fs=1000, output='sos')
    filtered = signal.sosfilt(sos, sig)
    ax2.plot(t, filtered)
    ax2.set_title(f'After {lower_freq_limit} Hz high-pass filter')
    ax2.axis([0, 1, -2, 2])
    ax2.set_xlabel('Time [seconds]')
v    plt.show()
    perform_fft(filtered, t)

def lp_filter(amplitudes, upper_freq_limit):
    sig = amplitudes
    fig, (ax2) = plt.subplots(1, 1, sharex=True)
    sos = signal.butter(10, upper_freq_limit, 'lp', fs=1000, output='sos')
    filtered = signal.sosfilt(sos, sig)
    ax2.plot(t, filtered)
    ax2.set_title(f'After {upper_freq_limit} Hz high-pass filter')
    ax2.axis([0, 1, -2, 2])
    ax2.set_xlabel('Time [seconds]')
    plt.tight_layout()
    plt.show()
    perform_fft(filtered, t)

hp_filter(sig,250)
lp_filter(sig,50)


In [ ]:
sos = signal.butter(10, 15, 'hp', fs=1000, output='sos')
